In [5]:
from dotenv import load_dotenv
import openai
import os
from openai import AzureOpenAI
from IPython.display import display, HTML, JSON, Markdown, Image

load_dotenv()
AZURE_OPENAI_ENDPOINT=os.getenv("AISTUDIO_AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_GPT4o_DEPLOYMENT=os.getenv("AI_STUDIO_AZURE_OPENAI_GPT4o_DEPLOYMENT")
AZURE_OPENAI_API_VERSION="2024-02-01"
AZURE_OPENAI_KEY=os.getenv("AISTUDIO_AZURE_OPENAI_KEY")
 
AZURE_COMPUTER_VISION_ENDPOINT = os.getenv("AZURE_COMPUTER_VISION_ENDPOINT")
AZURE_COMPUTER_VISION_KEY = os.getenv("AZURE_COMPUTER_VISION_KEY")

client = openai.AzureOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_KEY,
        api_version=AZURE_OPENAI_API_VERSION
)


In [2]:
import os
import json
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential

def analyze_image(image_filename):
    client = ImageAnalysisClient(
        endpoint=AZURE_COMPUTER_VISION_ENDPOINT,
        credential=AzureKeyCredential(AZURE_COMPUTER_VISION_KEY)
    )

    with open(image_filename, "rb") as image_stream:
        result = client.analyze(
            image_stream,
            visual_features=[VisualFeatures.CAPTION, VisualFeatures.READ, VisualFeatures.DENSE_CAPTIONS, VisualFeatures.OBJECTS, VisualFeatures.PEOPLE, VisualFeatures.TAGS, VisualFeatures.SMART_CROPS],
            gender_neutral_caption=True,  # Optional (default is False)
        )

    print("Image analysis results:")
    print(" Model version: {}".format(result.model_version))
    # Print caption results to the console
    if result.caption is not None:
        print(" Caption:")    
        print(f"   '{result.caption.text}', Confidence {result.caption.confidence:.4f}")
    if result.dense_captions is not None:
        print(" Dense Captions:")    
        for caption in result.dense_captions.list:
            print(f"   Text: '{caption.text}', Confidence: {caption.confidence}")
    if result.objects is not None:
        print(" Objects:")    
        for caption in result.objects.list:
            print(f"   Text: '{caption.text}', Confidence: {caption.confidence}")
    if result.tags is not None:
        print(" Tags:")
        for caption in result.tags.list:
            print(f"   Text: '{caption.name}', Confidence: {caption.confidence}")
    if result.people is not None:
        print(" People:")    
        print(f"   '{result.people.list}'")
    if result.smart_crops is not None:
        print(" Smart Crops:")    
        print(f"   '{result.smart_crops.list}'")
    
    result_dict = result.as_dict()
    result_json = json.dumps(result_dict)
    return result_json
            


#### Analyze a picture using Azure Cognitve services to extract text from a picture

In [3]:
image_url = "./data/retail/bike.jpg"
json_result = analyze_image(image_url)

Image analysis results:
 Model version: 2023-10-01
 Caption:
   'a blue and pink polo shirt', Confidence 0.7039
 Dense Captions:
   Text: 'a blue and pink polo shirt', Confidence: 0.7117427587509155
   Text: 'a blue shirt with purple and pink design', Confidence: 0.6390741467475891
   Text: 'a blue shirt with a tie', Confidence: 0.6828346848487854
 Objects:
 Tags:
   Text: 'clothing', Confidence: 0.999369740486145
   Text: 'fabric', Confidence: 0.9067329168319702
   Text: 'sleeve', Confidence: 0.9034119248390198
   Text: 'collar', Confidence: 0.8999518156051636
   Text: 'active shirt', Confidence: 0.8928235769271851
   Text: 'top', Confidence: 0.8922902345657349
   Text: 'shirt', Confidence: 0.8135350942611694
   Text: 'person', Confidence: 0.6144163608551025
 People:
   '[{'boundingBox': {'x': 103, 'y': 0, 'w': 113, 'h': 213}, 'confidence': 0.0026073618791997433}, {'boundingBox': {'x': 0, 'y': 0, 'w': 63, 'h': 223}, 'confidence': 0.0021513751707971096}, {'boundingBox': {'x': 0, 'y': 0

#### Extract all dense captions from the json result

In [4]:
import json
dict = json.loads(json_result)
dense_captions = dict["denseCaptionsResult"]["values"]


#### Generate a product description from the text extracted from the photo using OpenAI

In [7]:
def call_openAI(text):
    message_text = [
    {"role":"system","content":"You are an AI assistant that generates product descriptions for an online retail website especializing in clothing. Given the user's input, generate a Product Description."},
    {"role":"user","content":text}]

    response = client.chat.completions.create(
        model=AZURE_OPENAI_GPT4o_DEPLOYMENT,
        messages = message_text,
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    return response.choices[0].message.content

In [8]:
count = 1
for caption in dense_captions:
    result = call_openAI(caption["text"])
    print("Option " + str(count) + ": " + result)
    count = count + 1


Option 1: Elevate your casual wardrobe with our stylish Blue and Pink Polo Shirt. This versatile piece features a classic polo design with a modern twist, showcasing a vibrant blend of blue and pink hues that effortlessly catch the eye. Crafted from high-quality, breathable cotton fabric, this polo shirt ensures all-day comfort and durability. The ribbed collar and cuffs add a touch of sophistication, while the two-button placket allows for easy, adjustable wear. Perfect for a day out with friends, a casual office look, or a weekend getaway, this polo shirt pairs seamlessly with jeans, chinos, or shorts. Add a pop of color to your collection and make a statement with this must-have wardrobe essential.
Option 2: Elevate your wardrobe with this stunning blue shirt, featuring an intricate design in shades of purple and pink. Crafted from premium-quality fabric, this shirt offers both comfort and style, making it perfect for any occasion. The vibrant purple and pink patterns add a touch of